In [34]:
# Imports
import json
import numpy as np
import pandas as pd

# Load in the data file
with open('../question-data.json') as data_file:
    data = json.loads(data_file.read())

# Prepare the data
td_text = []
td_labels = []
str_to_num = {'injury': 0, 'trade': 1}
for index in data:
    td_text += [data[index][x]for x in range(len(data[index]))]
    td_labels += [str_to_num[index] for x in range(len(data[index]))]
#final = {"text": td_text, "label": td_labels}
final = [{"text": td_text[i], "label": td_labels[i], "idx": i} for i in range(len(td_text))]

In [35]:
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig
bert_tokenizer = BertTokenizer.from_pretrained("facebook/bart-large-mnli")
bert_model = TFBertForSequenceClassification.from_pretrained('facebook/bart-large-mnli',num_labels=2)

OSError: bart-base-uncased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
from transformers import BertTokenizer, TFBertModel, BertConfig
from sklearn.model_selection import train_test_split

input_ids=[]
attention_masks=[]

for sent in td_text:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(td_labels)

train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.4)

/Users/danielstefanescu/miniforge3/envs/mlp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
import tensorflow as tf

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

In [ ]:
print (train_label)

[1 0 1 1 0 0 0 1 0 1]


In [ ]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=4,validation_data=([val_inp,val_mask],val_label))

Epoch 1/4


2022-10-20 15:51:33.868814: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6073 - accuracy: 0.5000

2022-10-20 15:51:47.320448: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 23s 23s/step - loss: 0.6073 - accuracy: 0.5000 - val_loss: 0.5493 - val_accuracy: 1.0000
Epoch 2/4
1/1 [==============================] - 2s 2s/step - loss: 0.5572 - accuracy: 1.0000 - val_loss: 0.5444 - val_accuracy: 0.8750
Epoch 3/4
1/1 [==============================] - 1s 1s/step - loss: 0.5904 - accuracy: 0.8000 - val_loss: 0.4917 - val_accuracy: 1.0000
Epoch 4/4
1/1 [==============================] - 1s 873ms/step - loss: 0.5549 - accuracy: 0.7000 - val_loss: 0.4638 - val_accuracy: 1.0000


In [ ]:
def extract_predictions(text):
  a_preds = []
  a_scores = []
  data_encodings = bert_tokenizer(text, truncation=True, padding=True)
  ds = tf.data.Dataset.from_tensor_slices(dict(data_encodings)).batch(64)
  preds = bert_model.predict(ds)["logits"]
  classes = np.argmax(preds, axis=1).tolist()
  probs = tf.nn.softmax(preds) # get probabilities from logits
  scores = np.amax(probs, axis=1).tolist()

  a_preds.extend(classes)
  a_scores.extend(scores)
  # return predictions and scores
  ans = []
  num_to_str = {0: 'injury', 1: 'trade'}
  for i in range(len(a_preds)):  # map labels to label titles
    ans.append({
        "label": num_to_str[a_preds[i]],
        "score": a_scores[i]
    })
  ansdf = pd.DataFrame(ans)
  return ansdf

text = ["when will [player] be back?", "is [player] hurt?"]
result = extract_predictions(text)
print (result)

1/1 [==============================] - 1s 522ms/step
    label     score
0  injury  0.556148
1  injury  0.541180
2  injury  0.555820
